<a href="https://colab.research.google.com/github/jatinmayekar/ai_project_1/blob/main/TextQuery2PDFs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

README:
This Google Colab notebook is designed to load the OpenAI API key from a file stored in Google Drive. It sets up the necessary environment by installing required libraries and then reads the API key from a specified file in Google Drive. Once the key is loaded, it is set as an environment variable for further use in the notebook.

https://github.com/openai/tiktoken

https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb

https://pypi.org/search/?q=tiktoken

In [39]:
# Required libraries installation
!pip install langchain
!pip install unstructured[all-docs]
!pip install openai
!pip install chromadb
!pip install tiktoken

import os

# Importing necessary libraries for partitioning PDFs
from unstructured.partition.auto import partition
from unstructured.documents.elements import NarrativeText
from unstructured.partition.text_type import sentence_count

from langchain.text_splitter import NLTKTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI

In [40]:
from google.colab import drive
drive.mount('/content/drive')

# Adjust the path based on where you've placed the file in Google Drive
file_path = '/content/drive/My Drive/ai_portfolio/openai_key.txt'

with open(file_path, 'r') as file:
    api_key = file.readline().strip()

os.environ["OPENAI_API_KEY"] = api_key

print("API Key loaded successfully!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
API Key loaded successfully!


In [41]:
# Define the path to the PDF
link="/content/3HAC052982 PS IRB 14000-en.pdf"
# Partition the PDF content
elements = partition(filename=link, content_type="application/pdf")

input=""
for element in elements:
    if isinstance(element, NarrativeText):
        input=input+str(element);

nltksplit = NLTKTextSplitter(chunk_size=250)
nsplit = nltksplit.split_text(input)

In [42]:
embed = OpenAIEmbeddings()

db = Chroma.from_texts(nsplit, embed)

In [43]:
query = "what is yumi"
outputtext = db.similarity_search(query)
print(outputtext[0].page_content)

All rights reserved.1.1.1.1.1 Robot type description ContinuedWhen operated under the specified test conditions, the IRB 14000 YuMi including gripper and suction cup is suitable for use in cleanrooms fulfilling the specifications of the following Air Cleanliness Classes according to ISO 14644-1.The robot has IP30 protection.The robot is equipped with the controller (located inside the boby of the robot) and robot control software, RobotWare.


In [44]:
memory=ConversationBufferMemory(memory_key="chat_history",return_messages=True)

qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), db.as_retriever(), memory=memory)

In [46]:
query = "What is this manual about?"
result = qa({"question": query})
result["answer"]

' The safety instructions for the IRB 14000 manipulators and controllers are described in the Safety manual for robot - Manipulator and IRC5 or OmniCore controller. This manual contains all the necessary information for maintaining the safety integrity of the installation, including instructions for replacing units in the controller and reporting the data to ABB.'

In [47]:
query = "tell me how do I move my robot linearly and circularly"
result = qa({"question": query})
result["answer"]

" You can move your robot linearly and circularly by adjusting the robot's positioning, velocity, arm configuration, direction of approach, and load direction."

In [48]:
!pip install pypdf2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.7 MB/s eta 0:00:00


In [62]:
import PyPDF2

def pdf_to_text(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''.join([reader.pages[i].extract_text () for i in range(len(reader.pages))])
    return text


In [65]:
import re

def clean_text(text):
    # Remove any set of characters "XXX"
    text = text.replace("XXX", "")

    # Remove trailing whitespaces and filter lines
    cleaned_lines = []
    for line in text.splitlines():
        line = line.strip()  # Remove trailing whitespaces

        # Skip lines with non-standard characters
        if re.search(r"[^a-zA-Z0-9.,!?;:'\" \-()]", line):
            continue

        # Skip lines that are blank or only have periods
        if not line or line == ".":
            continue

        # If a line ends with a hyphen, remove that hyphen
        if line.endswith('-'):
            line = line[:-1]

        cleaned_lines.append(line)

    return '\n'.join(cleaned_lines)


In [66]:
pdf_path = "/content/3HAC052982 PS IRB 14000-en.pdf"
text = pdf_to_text(pdf_path)
cleaned_text = clean_text(text)

In [67]:
with open('/content/cleaned_text.txt', 'w') as file:
    file.write(cleaned_text)


In [68]:
    # Offer the PLY file for download.
    from google.colab import files
    files.download('cleaned_text.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [69]:
link="/content/cleaned_text.txt"
elements = partition(filename=link, content_type="application/text")

input=""
for element in elements:
    if isinstance(element, NarrativeText):
        input=input+str(element);

nltksplit = NLTKTextSplitter(chunk_size=250)
nsplit = nltksplit.split_text(input)

In [70]:
embed = OpenAIEmbeddings()

db = Chroma.from_texts(nsplit, embed)

In [91]:
query = "what is a YuMI"
outputtext = db.similarity_search(query)
print(outputtext[0].page_content)

All rights reserved.1.1.1.1.1 Robot type description ContinuedWhen operated under the specified test conditions, the IRB 14000 YuMi including gripper and suction cup is suitable for use in cleanrooms fulfilling the specifications of the following Air Cleanliness Classes according to ISO 14644-1.The robot has IP30 protection.The robot is equipped with the controller (located inside the boby of the robot) and robot control software, RobotWare.


In [92]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f" Show the initial query of the user and craft a refined answer based on the query: '{query}' and relevant chunk of text from the source pdf: '{outputtext[0].page_content}'?"}
    ]
)

refined_output = response.choices[0].message['content'].strip()
print(refined_output)


Query: "What is a YuMi?"

YuMi is a type of robot, specifically the IRB 14000 YuMi, that is suitable for use in cleanrooms. It has IP30 protection and is equipped with a controller and robot control software called RobotWare.
